### Look at encounter durations to check how long patients are in hospital, for both RRT & non-RRT encounters.
#### When this was first done, we did not use the checkin time from the checkin table, where appropriate. Fixed here.
#### Starts with looking at the durations for the data we used in model, for rrt & non-rrt. Then goes bigger picture, to show subsets of data.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from impala.util import as_pandas

In [ ]:
# connect to impala
from impala.dbapi import connect
conn = connect(host="mycluster.domain.com", port=my_impala_port_number)

In [ ]:
# Make sure we're pulling from the right location
cur = conn.cursor()
cur.execute('use my_db')


In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook
plt.style.use('ggplot')

In [ ]:
# Show tables to verify you're actually pulling from sandbox
cur.execute('SHOW TABLES')
cur.fetchall()

### Quick numbers: # RRT events & total # encounters (for the main hospital)
#### For all patient & location types

In [ ]:
query_TotalEncs = """
SELECT count(1) 
FROM ( 
   SELECT DISTINCT encntr_id
   FROM encounter 
   WHERE encntr_complete_dt_tm < 4000000000000 
   AND loc_facility_cd = '633867' 
   ) t;
"""

cur.execute(query_TotalEncs)
cur.fetchall()

#### For admit_type_cd!='0' & encntr_type_class_cd='391

In [ ]:
query_TotalEncs = """
SELECT count(1) 
FROM ( 
    SELECT DISTINCT encntr_id
    FROM encounter
    WHERE encntr_complete_dt_tm < 4e12
    AND loc_facility_cd = '633867'
    AND admit_type_cd!='0'
    AND encntr_type_class_cd='391'
    ) t;
"""

cur.execute(query_TotalEncs)
cur.fetchall()

### Examining distribution of encounter durations (with loc_facility_cd)
#### Analyze the durations of the RRT event patients.

In [ ]:
query_count = """
SELECT count(*) 
FROM (
    SELECT DISTINCT ce.encntr_id
    FROM clinical_event ce
    INNER JOIN encounter enc ON enc.encntr_id = ce.encntr_id
    WHERE ce.event_cd = '54411998'
    AND ce.result_status_cd NOT IN ('31', '36')
    AND ce.valid_until_dt_tm > 4e12
    AND ce.event_class_cd not in ('654645')
    AND enc.loc_facility_cd = '633867'
    AND enc.encntr_complete_dt_tm < 4e12
    AND enc.admit_type_cd!='0'
    AND enc.encntr_type_class_cd='391'
    ) AS A ;
"""
cur.execute(query_count)
cur.fetchall()

In [ ]:
query_count = """
SELECT count(*) 
FROM (
    SELECT DISTINCT encntr_id
    FROM encounter enc
    WHERE enc.loc_facility_cd = '633867'
    AND enc.encntr_complete_dt_tm < 4e12
    AND enc.admit_type_cd!='0'
    AND enc.encntr_type_class_cd='391'
    AND encntr_id NOT IN (
                        SELECT DISTINCT ce.encntr_id
                        FROM clinical_event ce
                        INNER JOIN encounter enc ON enc.encntr_id = ce.encntr_id
                        WHERE ce.event_cd = '54411998'
                        AND ce.result_status_cd NOT IN ('31', '36')
                        AND ce.valid_until_dt_tm > 4e12
                        AND ce.event_class_cd not in ('654645')
                        )
    ) AS A;
"""
cur.execute(query_count)
cur.fetchall()

### Let's look at durations for inpatients WITH RRTs from the Main Hospital where encounter_admit_type is not zero

In [ ]:
query = """
SELECT 
      DISTINCT ce.encntr_id 
    , COALESCE(tci.checkin_dt_tm, enc.arrive_dt_tm) AS checkin_dt_tm
    , enc.depart_dt_tm as depart_dt_tm
    , (enc.depart_dt_tm - COALESCE(tci.checkin_dt_tm, enc.arrive_dt_tm))/3600000 AS diff_hours
    , enc.reason_for_visit
    , enc.admit_src_cd
    , enc.admit_type_cd
FROM clinical_event ce 
INNER JOIN encounter enc ON enc.encntr_id = ce.encntr_id
LEFT OUTER JOIN  ( 
                SELECT  
                      ti.encntr_id AS encntr_id
                    , MIN(tc.checkin_dt_tm) AS checkin_dt_tm
                FROM tracking_item ti
                JOIN tracking_checkin tc ON ti.tracking_id = tc.tracking_id
                GROUP BY ti.encntr_id 
                 ) tci
ON tci.encntr_id = enc.encntr_id
WHERE enc.loc_facility_cd = '633867'
AND enc.encntr_complete_dt_tm < 4e12
AND enc.admit_type_cd!='0'
AND enc.encntr_type_class_cd='391'
AND enc.encntr_id IN ( 
                    SELECT DISTINCT ce.encntr_id
                    FROM clinical_event ce
                    WHERE ce.event_cd = '54411998'
                    AND ce.result_status_cd NOT IN ('31', '36')
                    AND ce.valid_until_dt_tm > 4e12
                    AND ce.event_class_cd not in ('654645') 
                    )
;"""

In [ ]:
cur.execute(query)
df_rrt = as_pandas(cur)

In [ ]:
df_rrt.head()

In [ ]:
df_rrt.describe().T
# the mean stay is 292 hours (12.1 days).
# The median stay is 184 hours (7.67 days)
# The minimum stay is 8 hours. The longest stay is 3550 hours (~148 days)

In [ ]:
plt.figure()
df_rrt.diff_hours.hist(bins = 300)
plt.xlim(0, 600)

In [ ]:
# Records with short durations:
df_rrt[df_rrt.diff_hours < 12]

### Let's look at durations for inpatients WITHOUT RRTs from the Main Hospital where encounter_admit_type is not zero

In [ ]:
query = """
SELECT DISTINCT 
      ce.encntr_id
    , COALESCE(tci.checkin_dt_tm
    , enc.arrive_dt_tm) AS checkin_dt_tm
    , enc.depart_dt_tm as depart_dt_tm
    , (enc.depart_dt_tm - COALESCE(tci.checkin_dt_tm, enc.arrive_dt_tm))/3600000 AS diff_hours
    , enc.reason_for_visit
    , enc.admit_src_cd
    , enc.admit_type_cd
FROM clinical_event ce 
INNER JOIN encounter enc ON enc.encntr_id = ce.encntr_id
LEFT OUTER JOIN  ( 
                SELECT  
                      ti.encntr_id AS encntr_id
                    , MIN(tc.checkin_dt_tm)  AS checkin_dt_tm
                FROM tracking_item ti
                JOIN tracking_checkin tc ON ti.tracking_id = tc.tracking_id
                GROUP BY ti.encntr_id 
                ) tci
ON tci.encntr_id = enc.encntr_id
WHERE enc.loc_facility_cd = '633867'
AND enc.encntr_complete_dt_tm < 4e12
AND enc.admit_type_cd!='0'
AND enc.encntr_type_class_cd='391'
AND enc.encntr_id NOT IN ( 
                        SELECT DISTINCT ce.encntr_id
                        FROM clinical_event ce
                        WHERE ce.event_cd = '54411998'
                        AND ce.result_status_cd NOT IN ('31', '36')
                        AND ce.valid_until_dt_tm > 4e12
                        AND ce.event_class_cd not in ('654645') 
                        )
;"""

In [ ]:
cur.execute(query)
df_nonrrt = as_pandas(cur)

In [ ]:
df_nonrrt.describe().T
# NonRRT: The mean stay is 122 hours (5 days) // RRT: The mean stay is 292 hours (12.1 days).
# NonRRT: The median stay is 77 hours (3.21 days)// RRT: The median stay is 184 hours (7.67 days)
# NonRRT: The minimum stay is 0.08 hours // RRT: The minimum stay is ~8 hours.

In [ ]:
plt.figure()
df_nonrrt.diff_hours.hist(bins = 500)
plt.xlim(0, 600)

### Plot both together to see how encounter duration distributions are different

In [ ]:
plt.figure(figsize = (10,8))
df_rrt.diff_hours.plot.hist(alpha=0.4, bins=400,normed=True)
df_nonrrt.diff_hours.plot.hist(alpha=0.4, bins=800,normed=True)
plt.xlabel('Hospital Stay Durations, hours', fontsize=14)
plt.ylabel('Normalized Frequency', fontsize=14)
plt.legend(['RRT', 'Non RRT'])
plt.tick_params(labelsize=14)
plt.xlim(0, 1000)

## Even accounting for the hospital, inpatients status, and accounting for some admit_type_cd, the durations are still quite different betwen RRT & non-RRT.
### Trying some subset vizualizations -- these show no difference

In [ ]:
print df_nonrrt.admit_type_cd.value_counts()
print
print df_rrt.admit_type_cd.value_counts()

In [ ]:
print df_nonrrt.admit_src_cd.value_counts()
print
print df_rrt.admit_src_cd.value_counts()

In [ ]:
plt.figure(figsize = (10,8))
df_rrt[df_rrt.admit_type_cd=='309203'].diff_hours.plot.hist(alpha=0.4, bins=300,normed=True)
df_nonrrt[df_nonrrt.admit_type_cd=='309203'].diff_hours.plot.hist(alpha=0.4, bins=600,normed=True)
# plt.xlabel('Hospital Stay Durations, hours', fontsize=14)
# plt.ylabel('Normalized Frequency', fontsize=14)
plt.legend(['RRT', 'Non RRT'])
plt.tick_params(labelsize=14)
plt.xlim(0, 1000)

In [ ]:
plt.figure(figsize = (10,8))
df_rrt[df_rrt.admit_src_cd=='309196'].diff_hours.plot.hist(alpha=0.4, bins=300,normed=True)
df_nonrrt[df_nonrrt.admit_src_cd=='309196'].diff_hours.plot.hist(alpha=0.4, bins=600,normed=True)
# plt.xlabel('Hospital Stay Durations, days', fontsize=14)
# plt.ylabel('Normalized Frequency', fontsize=14)
plt.legend(['RRT', 'Non RRT'])
plt.tick_params(labelsize=14)
plt.xlim(0, 1000)

# Despite controlling for patient parameters, patients with RRT events stay in the hospital longer than non-RRT event having patients.

## Rerun previous EDA on hospital & patient types

### Let's take a step back and look at the encounter table, for all hospitals and patient types [but using corrected time duration].

In [ ]:
# For encounters with RRT events
query = """
SELECT DISTINCT 
      ce.encntr_id
    , COALESCE(tci.checkin_dt_tm
    , enc.arrive_dt_tm) AS checkin_dt_tm
    , enc.depart_dt_tm as depart_dt_tm
    , (enc.depart_dt_tm - COALESCE(tci.checkin_dt_tm, enc.arrive_dt_tm))/3600000 AS diff_hours
    , enc.reason_for_visit
    , enc.admit_type_cd, cv_admit_type.description as admit_type_desc
    , enc.encntr_type_cd
    , cv_enc_type.description as enc_type_desc
    , enc.encntr_type_class_cd
    , cv_enc_type_class.description as enc_type_class_desc
    , enc.admit_src_cd
    , cv_admit_src.description as admit_src_desc
    , enc.loc_facility_cd
    , cv_loc_fac.description as loc_desc
FROM clinical_event ce 
INNER JOIN encounter enc ON enc.encntr_id = ce.encntr_id
LEFT OUTER JOIN code_value cv_admit_type ON enc.admit_type_cd = cv_admit_type.code_value
LEFT OUTER JOIN code_value cv_enc_type ON enc.encntr_type_cd = cv_enc_type.code_value
LEFT OUTER JOIN code_value cv_enc_type_class ON enc.encntr_type_class_cd = cv_enc_type_class.code_value
LEFT OUTER JOIN code_value cv_admit_src ON enc.admit_src_cd = cv_admit_src.code_value
LEFT OUTER JOIN code_value cv_loc_fac ON enc.loc_facility_cd = cv_loc_fac.code_value
LEFT OUTER JOIN  ( 
                SELECT  
                      ti.encntr_id AS encntr_id
                    , MIN(tc.checkin_dt_tm) AS checkin_dt_tm
                FROM tracking_item ti
                JOIN tracking_checkin tc ON ti.tracking_id = tc.tracking_id
                GROUP BY ti.encntr_id 
                ) tci
ON tci.encntr_id = enc.encntr_id
WHERE enc.encntr_id IN ( 
                    SELECT DISTINCT ce.encntr_id
                    FROM clinical_event ce
                    WHERE ce.event_cd = '54411998'
                    AND ce.result_status_cd NOT IN ('31', '36')
                    AND ce.valid_until_dt_tm > 4e12
                    AND ce.event_class_cd not in ('654645') 
                    )
;"""

In [ ]:
cur.execute(query)
df = as_pandas(cur)

In [ ]:
df.describe().T

In [ ]:
# check nulls
print df[pd.isnull(df.diff_hours)].count()
print 
print df[~pd.isnull(df.diff_hours)].count()

In [ ]:
df[pd.isnull(df.diff_hours)]

In [ ]:
# can't work with the nans in there... delete these rows
print df.shape
df = df[~pd.isnull(df['depart_dt_tm'])]
df = df.reset_index(drop=True)
print df.shape

In [ ]:
df.describe().T
# RRT encounters for all patients/hospitals
# All RRT: mean stay: 293.5 hours // NonRRT: The mean stay is 122 hours (5 days) // RRT: The mean stay is 292 hours (12.1 days).
# All RRT: median stay: 190 hours // NonRRT: The median stay is 77 hours (3.21 days)// RRT: The median stay is 184 hours (7.67 days)
# All RRT: min stay: 0 hours // NonRRT: The minimum stay is 0.08 hours // RRT: The minimum stay is ~8 hours.

In [ ]:
# Let's be suspicious of short encounters, say, under 6 hours.
# There are two cases where the number of hours = 0, these both have admit_type_cd=0, loc_facility_cd = 4382287. & ecntr_type_class_cd=393
df[df.diff_hours < 6]

### The notebook Probe_encounter_types_classes explores admit type, class types & counts

In [ ]:
plt.figure()
df['diff_hours'].plot.hist(bins=500)
plt.xlabel("Hospital Stay Duration, days")
plt.title("Range of stays, patients with RRT")
plt.xlim(0, 2000)

## Group by facility
### We want to pull from similar patient populations

In [ ]:
df.head()

In [ ]:
df.loc_desc.value_counts()

In [ ]:
grouped = df.groupby('loc_desc')

In [ ]:
grouped.describe()

### Most number of results from 633867, or The Main Hospital

In [ ]:
df.diff_hours.hist(by=df.loc_desc, bins=300)

In [ ]:
# Use locations 4382264, 4382273, 633867

In [ ]:
plt.figure(figsize=(12, 6))
df[df['loc_facility_cd']=='633867']['diff_hours'].plot.hist(alpha=0.4, bins=300,normed=True)
df[df['loc_facility_cd']=='4382264']['diff_hours'].plot.hist(alpha=0.4, bins=300,normed=True)
df[df['loc_facility_cd']=='4382273']['diff_hours'].plot.hist(alpha=0.4, bins=300,normed=True)
plt.xlabel('Hospital Stay Durations, days', fontsize=14)
plt.ylabel('Normalized Frequency', fontsize=14)
# plt.legend(['633867', '4382264', '4382273'])
plt.legend(["Main Hospital", "Sattelite Hospital 1", "Sattelite Hospital 2"])
plt.tick_params(labelsize=14)
plt.xlim(0, 1000)

## Looks like these three locations (633867, 4382264, 4382273) have about the same distribution. 
#### Appropriate test to verify this: 2-sample Kolmogorov-Smirnov, if you're willing to compare pairwise...other tests? Wikipedia has a good article with references: https://en.wikipedia.org/wiki/Kolmogorov–Smirnov_test. Null hypothesis: the samples come from the same distribution. The null hypothesis is rejected if the test statistic is greater than the critical value (see wiki article)


In [ ]:
from scipy.stats import ks_2samp

In [ ]:
ks_2samp(df[df['loc_facility_cd']=='633867']['diff_hours'],df[df['loc_facility_cd']=='4382264']['diff_hours'])

In [ ]:
# Critical test statistic at alpha = 0.05: = 1.36 * sqrt((n1+n2)/n1*n2) = 1.36*(sqrt((1775+582)/(1775*582)) = 0.065
# 0.074 > 0.065 -> null hypothesis rejected at level 0.05. --> histograms are different

In [ ]:
ks_2samp(df[df['loc_facility_cd']=='4382264']['diff_hours'], df[df['loc_facility_cd']=='4382273']['diff_hours'])

In [ ]:
# Critical test statistic at alpha = 0.05: = 1.36 * sqrt((n1+n2)/n1*n2) = 1.36*(sqrt((997+582)/(997*582)) = 0.071
# 0.05 !> 0.071 -> fail to reject null hypothesis at level 0.05. --> histograms are similar

In [ ]:
ks_2samp(df[df['loc_facility_cd']=='633867']['diff_hours'],df[df['loc_facility_cd']=='4382273']['diff_hours'])

In [ ]:
# Critical test statistic at alpha = 0.05: = 1.36 * sqrt((n1+n2)/n1*n2) = 1.36*(sqrt((1775+997)/(1775*997)) = 0.054
# 0.094 > 0.054 -> null hypothesis rejected at level 0.05. --> histograms are different; p-value indicates they're very different

#### From scipy documentation: "If the KS statistic is small or the p-value is high, then we cannot reject the hypothesis that the distributions of the two samples are the same"
Null hypothesis: the distributions are the same.
Looks like samples from 4382273 are different... plot that & 633867

In [ ]:
plt.figure(figsize=(10,8))
df[df['loc_facility_cd']=='633867']['diff_hours'].plot.hist(alpha=0.4, bins=500,normed=True)
df[df['loc_facility_cd']=='4382273']['diff_hours'].plot.hist(alpha=0.4, bins=700,normed=True)
plt.xlabel('Hospital Stay Durations, hours')
plt.legend(['633867', '4382273'])
plt.xlim(0, 1000)

# Let's compare encounter duration histograms for patients with RRT & without RRT events, and see if there is a right subset of data to be selected for modeling
### (There is)

In [ ]:
df.columns

In [ ]:
df.admit_src_desc.value_counts()

In [ ]:
df.enc_type_class_desc.value_counts()
# vast majority are inpatient

In [ ]:
df.enc_type_desc.value_counts()

In [ ]:
df.admit_type_desc.value_counts()

## Plot RRT & non-RRT with different codes

In [ ]:
# For encounters without RRT events, from Main Hospital.
# takes a while to run -- several minutes
query = """
SELECT DISTINCT 
      ce.encntr_id
    , COALESCE(tci.checkin_dt_tm
    , enc.arrive_dt_tm) AS checkin_dt_tm
    , enc.depart_dt_tm as depart_dt_tm
    , (enc.depart_dt_tm - COALESCE(tci.checkin_dt_tm, enc.arrive_dt_tm))/3600000 AS diff_hours
    , enc.reason_for_visit
    , enc.admit_type_cd
    , cv_admit_type.description as admit_type_desc
    , enc.encntr_type_cd
    , cv_enc_type.description as enc_type_desc
    , enc.encntr_type_class_cd
    , cv_enc_type_class.description as enc_type_class_desc
    , enc.admit_src_cd
    , cv_admit_src.description as admit_src_desc
    , enc.loc_facility_cd
    , cv_loc_fac.description as loc_desc
FROM clinical_event ce 
INNER JOIN encounter enc ON enc.encntr_id = ce.encntr_id
LEFT OUTER JOIN code_value cv_admit_type ON enc.admit_type_cd = cv_admit_type.code_value
LEFT OUTER JOIN code_value cv_enc_type ON enc.encntr_type_cd = cv_enc_type.code_value
LEFT OUTER JOIN code_value cv_enc_type_class ON enc.encntr_type_class_cd = cv_enc_type_class.code_value
LEFT OUTER JOIN code_value cv_admit_src ON enc.admit_src_cd = cv_admit_src.code_value
LEFT OUTER JOIN code_value cv_loc_fac ON enc.loc_facility_cd = cv_loc_fac.code_value
LEFT OUTER JOIN  ( 
                SELECT  
                      ti.encntr_id AS encntr_id
                    , MIN(tc.checkin_dt_tm) AS checkin_dt_tm
                FROM tracking_item ti
                JOIN tracking_checkin tc ON ti.tracking_id = tc.tracking_id
                GROUP BY ti.encntr_id 
                ) tci
ON tci.encntr_id = enc.encntr_id
WHERE enc.loc_facility_cd='633867'
AND enc.encntr_id NOT IN ( 
                        SELECT DISTINCT ce.encntr_id
                        FROM clinical_event ce
                        WHERE ce.event_cd = '54411998'
                        AND ce.result_status_cd NOT IN ('31', '36')
                        AND ce.valid_until_dt_tm > 4e12
                        AND ce.event_class_cd not in ('654645') 
                        )
;"""
cur.execute(query)
df_nrrt = as_pandas(cur)

In [ ]:
df_nrrt.describe()

In [ ]:
df_nrrt[~pd.isnull(df_nrrt['depart_dt_tm'])].count()

In [ ]:
# can't work with the nans in there... delete these rows
print df_nrrt.shape
df_nrrt = df_nrrt[~pd.isnull(df_nrrt['depart_dt_tm'])]
df_nrrt = df_nrrt.reset_index(drop=True)
print df_nrrt.shape

In [ ]:
plt.figure(figsize=(10,8))
df[df['loc_facility_cd']=='633867']['diff_hours'].plot.hist(alpha=0.5, bins=500,normed=True)
df_nrrt['diff_hours'].plot.hist(alpha=0.5, bins=900,normed=True)
plt.xlabel('Stay Durations at Main Hospital [hours]')
plt.legend(['RRT patients', 'Non-RRT patients'])
plt.title('For all non-RRT patients')
plt.xlim(0, 800)

In [ ]:
plt.figure(figsize=(10,8))
df[df['loc_facility_cd']=='633867']['diff_hours'][df.admit_type_cd != '0'].plot.hist(alpha=0.5, bins=500,normed=True)
df_nrrt['diff_hours'][df_nrrt.admit_type_cd != '0'].plot.hist(alpha=0.5, bins=900,normed=True)
plt.xlabel('Stay Durations at Main Hospital [hours]')
plt.legend(['RRT patients', 'Non-RRT patients'])
plt.title('For patients with admit_type_cd !=0')
plt.xlim(0, 800)

In [ ]:
plt.figure(figsize=(10,8))
df[df['loc_facility_cd']=='633867']['diff_hours'][df.encntr_type_class_cd=='391'].plot.hist(alpha=0.5, bins=500,normed=True)
df_nrrt['diff_hours'][df_nrrt.encntr_type_class_cd=='391'].plot.hist(alpha=0.5, bins=900,normed=True)
plt.xlabel('Stay Durations at Main Hospital [hours]')
plt.legend(['RRT patients', 'Non-RRT patients'])
plt.title('For patients with encntr_type_class_cd=="391"')
plt.xlim(0, 800)

In [ ]:
plt.figure(figsize=(10,8))
df[df['loc_facility_cd']=='633867']['diff_hours'][(df.encntr_type_class_cd=='391') & (df.admit_type_cd != '0')].plot.hist(alpha=0.5, bins=500,normed=True)
df_nrrt['diff_hours'][(df_nrrt.encntr_type_class_cd=='391') & (df_nrrt.admit_type_cd != '0')].plot.hist(alpha=0.5, bins=1000,normed=True)
plt.xlabel('Stay Durations at Main Hospital [hours]')
plt.legend(['RRT patients', 'Non-RRT patients'])
plt.title('For patients with encntr_type_class_cd=="391" & df.admit_type_cd != "0" ')
plt.xlim(0, 800)

In [ ]:
df_nrrt.describe()

In [ ]:
# There are values of diff_hours that are negative.
df_nrrt[df_nrrt.diff_hours<0].count()

In [ ]:
# But, there are no such values after we correct for encounter type class & admit type
df_nrrt[(df_nrrt.encntr_type_class_cd=='391') & (df_nrrt.admit_type_cd != '0')][df_nrrt.diff_hours<0].count()